# 202: Exampville Mode Choice Logsums

Welcome to Exampville, the best simulated town in this here part of the internet!

Exampville is a demonstration provided with Larch that walks through some of the 
data and tools that a transportation planner might use when building a travel model. 

In [ ]:
# TEST
from pytest import approx

import larch as lx

In [ ]:
import numpy as np
import xarray as xr
from addicty import Dict

import larch as lx

In this example notebook, we will walk through the creation of logsums from
an existing tour mode choice model.  First, let's load the data files from
our example.

In [ ]:
hh, pp, tour, skims = lx.example(200, ["hh", "pp", "tour", "skims"])

We'll also load the saved model from the mode choice estimation.

In [ ]:
exampville_mode_choice_file = lx.example(
    201, output_file="exampville_mode_choice.html", estimate=True
)
m = lx.load_model(exampville_mode_choice_file)

We'll replicate the pre-processing used in the mode choice estimation,
to merge the household and person characteristics into the tours data,
add the index values for the home TAZ's, filter to include only 
work tours, and merge with the level of service skims.  (If this 
pre-processing was computationally expensive, it would probably have
been better to save the results to disk and reload them as needed,
but for this model these commands will run almost instantaneously.)

In [ ]:
Mode = Dict(
    DA=1,
    SR=2,
    Walk=3,
    Bike=4,
    Transit=5,
).freeze()

tour_dataset = lx.Dataset.construct.from_idco(tour.set_index("TOURID"), alts=Mode)
od_skims = lx.Dataset.construct.from_omx(skims)

dt = lx.DataTree(
    tour=tour_dataset.dc.query_cases("TOURPURP == 1"),
    hh=hh.set_index("HHID"),
    person=pp.set_index("PERSONID"),
    od=od_skims,
    do=od_skims,
    relationships=(
        "tours.HHID @ hh.HHID",
        "tours.PERSONID @ person.PERSONID",
        "hh.HOMETAZ @ od.otaz",
        "tours.DTAZ @ od.dtaz",
        "hh.HOMETAZ @ do.dtaz",
        "tours.DTAZ @ do.otaz",
    ),
)

Then we bundle the raw data into the `larch.DataFrames` structure,
as we did for estimation, and attach this structure to the model
as its `dataservice`.

We'll also initialize a DataArray to hold the computed logsums.
This data will have one row for each case in our source data,
and a column for each possible destination zone.

In [ ]:
logsums = lx.DataArray.zeros(
    dt.caseids(),
    skims.TAZ_ID,
    name="logsums",
)

The logsums from a Model can be computed using the `Model.logsums` method.
However, if we want the logsums for each possible destination, we'll need
to replace the part of our data that depends on the destination zone, 
writing in the appropriate values for each.  We can simply iterate over the
zones, dropping in the zone as the destination and computing the logsums.

In [ ]:
for dtaz in logsums.TAZ_ID:
    m.datatree = dt.replace_datasets(
        tour=dt.root_dataset.assign(DTAZ=xr.full_like(dt._getitem("DTAZ"), dtaz)),
    )
    logsums.loc[dict(TAZ_ID=dtaz)] = m.logsums()

In [ ]:
# TEST
assert logsums[:4, :5].values == approx(
    np.array(
        [
            [-2.935642, -2.660468, -1.745647, -2.229293, -2.403132],
            [-2.935642, -2.660468, -1.745647, -2.229293, -2.403132],
            [-2.935642, -2.660468, -1.745647, -2.229293, -2.403132],
            [-3.046127, -2.770006, -1.823385, -2.320373, -2.4941],
        ]
    ),
    rel=1e-3,
)
assert logsums.shape == (7564, 40)
assert logsums[-4:, -5:].values == approx(
    np.array(
        [
            [-0.354058, -0.729078, -1.164068, -0.421579, -0.913035],
            [-0.354058, -0.729078, -1.164068, -0.421579, -0.913035],
            [-0.246731, -0.721721, -1.156756, -0.354397, -0.890595],
            [-0.391292, -0.731449, -1.166416, -0.444101, -0.920291],
        ]
    ),
    rel=1e-3,
)

Then we can persist the logsums dataframe to disk, for use in the next
example, where we will estimate a destination choice model.

In [ ]:
# TEST
import os
import shutil

if os.path.isdir("logsums.zarr"):
    shutil.rmtree("logsums.zarr")

In [ ]:
logsums.to_zarr("logsums.zarr")

To recover the DataArray later, we can read it using the `DataArray.from_zarr` method.

In [ ]:
lx.DataArray.from_zarr("logsums.zarr")

In [ ]:
# TEST
reloaded = lx.DataArray.from_zarr("logsums.zarr")
assert reloaded[:4, :5].values == approx(
    np.array(
        [
            [-2.935642, -2.660468, -1.745647, -2.229293, -2.403132],
            [-2.935642, -2.660468, -1.745647, -2.229293, -2.403132],
            [-2.935642, -2.660468, -1.745647, -2.229293, -2.403132],
            [-3.046127, -2.770006, -1.823385, -2.320373, -2.4941],
        ]
    ),
    rel=1e-3,
)
assert reloaded.shape == (7564, 40)
assert reloaded[-4:, -5:].values == approx(
    np.array(
        [
            [-0.354058, -0.729078, -1.164068, -0.421579, -0.913035],
            [-0.354058, -0.729078, -1.164068, -0.421579, -0.913035],
            [-0.246731, -0.721721, -1.156756, -0.354397, -0.890595],
            [-0.391292, -0.731449, -1.166416, -0.444101, -0.920291],
        ]
    ),
    rel=1e-3,
)